<center>
    
# Assignment-1

### Pooja Bandal 
####  2022-02-06 
 </center>

#### To install nltk and pytrec-terrier libraries

In [35]:
pip install nltk
pip install "pytrec-eval-terrier"

SyntaxError: invalid syntax (Temp/ipykernel_11240/2658067184.py, line 1)

In [ ]:
import concurrent.futures
import json
import nltk
import os
import pytrec_eval
import time
import tqdm

from utils.common_utils import return_top_1_5_10_words, return_success_at_k, find_closes_match


#### download the wordnet corpus

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
result_dir = r'Result'
os.makedirs(result_dir, exist_ok=True)


####  create a lists to store correct and incorrect spellings of words from missp.dat file

In [ ]:
correct_spellings = []
incorrect_spellings = []
file_ = open('Data/missp.dat', 'r')
Lines = file_.readlines()


#### Strips the newline character

In [36]:
crr_spell = ''
icrr_spell = ''
for line in Lines:
    if '$' in line:
        crr_spell = line.replace('$', '').replace('\n', '').lower()
    else:
        incrr_spell = line.lower().replace('\n', '')
        correct_spellings.append(crr_spell)
        incorrect_spellings.append(incrr_spell)

In [42]:
len(incorrect_spellings)

72266

#### finiding matching words using MED for 4 words without parallelization

In [28]:
incorrect_words = ['caugt', 'nit', 'siit', 'garl']
start = time.time()
results = []
for word in tqdm.tqdm(incorrect_words, desc='Without parallelization'):
    results.append(find_closes_match(word))
print('#' * 60)
print(f'Without parallelization the time taken is {round(time.time()-start, 4)} second(s)')
print('#' * 60)
del results
print('\n')

Without parallelization: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [03:31<00:00, 52.84s/it]


############################################################
Without parallelization the time taken is 211.3826 second(s)
############################################################




#### finding matching words using MED for 4 words with parallelization

#####  

In [29]:
start = time.time()
results = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    results.append(executor.map(find_closes_match, incorrect_words))
print('#' * 60)
print(f'With parallelization the time taken is {round(time.time() - start, 4)} second(s)')
print('#' * 60)
del results
print('\n')

############################################################
With parallelization the time taken is 103.3952 second(s)
############################################################




In [47]:
results = []
argument_list = [(icrr, crr) for icrr, crr in zip(incorrect_spellings, correct_spellings)]

with concurrent.futures.ProcessPoolExecutor() as executor:
    for result in executor.map(return_top_1_5_10_words, argument_list[:10]):
        results.append(result)

In [48]:
results

[{'incorrect': 'ab',
  'correct': 'albert',
  1: ['ab'],
  5: ['ab',
   'fab',
   'a',
   'aba',
   'abb',
   'abc',
   'abm',
   'abo',
   'abs',
   'alb',
   'arb',
   'b',
   'cab',
   'dab',
   'gab',
   'jab',
   'lab',
   'tab',
   'aby',
   'nab'],
  10: ['ab',
   'fab',
   'a',
   'aba',
   'abb',
   'abc',
   'abm',
   'abo',
   'abs',
   'alb',
   'arb',
   'b',
   'cab',
   'dab',
   'gab',
   'jab',
   'lab',
   'tab',
   'aby',
   'nab']},
 {'incorrect': 'ameraca',
  'correct': 'america',
  1: ['america'],
  5: ['america',
   'american',
   'arauca',
   'arca',
   'asmera',
   'camera',
   'maraca',
   'amerce'],
  10: ['america',
   'american',
   'arauca',
   'arca',
   'asmera',
   'camera',
   'maraca',
   'amerce',
   'amber',
   'aaa',
   'abaca',
   'aceraceae',
   'aec',
   'amberjack',
   'ameba',
   'ameer',
   'americana',
   'ametria',
   'amora',
   'ara',
   'araceae',
   'arava',
   'arc',
   'areca',
   'armeria',
   'armoracia',
   'camera_care',
   'camer

In [40]:
del argument_list

In [41]:
success_at_k = return_success_at_k(results)

query = {}
results_eval = {}
for result in results:
    query[result["incorrect"]] = {result["correct"]: 1}
    results_eval[result["incorrect"]] = {}
    for word in result[1]:
        results_eval[result["incorrect"]][word] = 1

    for word in result[5]:
        if word not in results_eval[result["incorrect"]].keys():
            results_eval[result["incorrect"]][word] = 1/5

    for word in result[10]:
        if word not in results_eval[result["incorrect"]].keys():
            results_eval[result["incorrect"]][word] = 1/10

evaluator = pytrec_eval.RelevanceEvaluator(query, {'success'})

print(json.dumps(evaluator.evaluate(results_eval), indent=1))
eval = evaluator.evaluate(results_eval)

for measure in sorted(list(eval[list(eval.keys())[0]].keys())):
    print(measure, 'average:',
          pytrec_eval.compute_aggregated_measure(
              measure, [query_measures[measure] for query_measures in eval.values()])
          )

{
 "ab": {
  "success_1": 0.0,
  "success_5": 0.0,
  "success_10": 0.0
 },
 "ameraca": {
  "success_1": 1.0,
  "success_5": 1.0,
  "success_10": 1.0
 },
 "amercia": {
  "success_1": 0.0,
  "success_5": 1.0,
  "success_10": 1.0
 },
 "ameracan": {
  "success_1": 1.0,
  "success_5": 1.0,
  "success_10": 1.0
 },
 "apirl": {
  "success_1": 0.0,
  "success_5": 1.0,
  "success_10": 1.0
 }
}
success_1 average: 0.26672162034856334
success_10 average: 0.47850918511540275
success_5 average: 0.41371290626471974
